The purpose of this notebook is to list various methods to compute the equation of the hyperplane representing a Voronoi ridge in any dimension. _These methods were found from a preliminary survey for methods to address this need and should be expanded on._ (see `TODO.md`)

Computing a hyperplane representation of a ridge will be NECESSARY (see `TODO.md`) for encoding ridges (in tessellations of dimension greater than 2D) as properties of queries to Marabou. The current implementation only accounts for 2D ridges and needs to be improved to scale to higher-dimensional ridges.

The below are some ways to implement this hyperplane representation computation that each have their own limitations.

In [1]:
import string
import warnings
warnings.filterwarnings('ignore')

import numpy as np

def pretty_print_hyperplane(coeffs, const=1.0):
    for i in range(len(coeffs)):
        suffix = " + " if (i < len(coeffs) - 1) else " "
        print(f"{coeffs[i]:.4f}*{string.ascii_lowercase[i]}", end=suffix)
    print(f"= {const}")

##### Method 1: Solving Linear System with Inverse
**Reference:** https://stackoverflow.com/questions/36270116/how-do-i-define-a-hyperplane-in-python-given-4-points-how-do-i-then-define-the

**Limitation:** Given points that form a singular matrix, a hyperplane equation cannot be computed. Would need to find a way to choose one of many possible hyperplanes in singular cases.

In [2]:
def hyper4(p1,p2,p3,p4):
    X=np.matrix([p1,p2,p3,p4])
    k=np.ones((4,1))
    
    try:
        return np.matrix.dot(np.linalg.inv(X), k)
    except:
        print("Cannot compute hyperplane equation (singular 4D matrix given)")

no_solution = hyper4([0,0,0,0],[0,0,0,1],[0,0,2,1],[0,0,2,0])  # No solution returned (singular 4D matrix)
a = hyper4([1,2,3,4],[0,0,0,1],[4,3,2,1],[2,0,2,0])  # invertible 4D: 1.15w - 1.1x - 0.65y + z = 1
print(f"Hyperplane: {a[0, 0]}*x1 + {a[1, 0]}*x2 + {a[2, 0]}*x3 + {a[3, 0]}*x4 = 1")

def hyper3(p1,p2,p3):
    X=np.matrix([p1,p2,p3])
    k=np.ones((4,1))

    try:
        return np.matrix.dot(np.linalg.inv(X), k)
    except:
        print("Cannot compute hyperplane equation (singular 3D matrix given)")

no_solution = hyper3([1,0,0],[0,1,0],[2,0,0])  # No solution returned (singular 3D matrix)

Cannot compute hyperplane equation (singular 4D matrix given)
Hyperplane: 1.15*x1 + -1.1*x2 + -0.6500000000000001*x3 + 1.0*x4 = 1
Cannot compute hyperplane equation (singular 3D matrix given)


##### Method 2: Solving Linear System by Pre-multiplying Both Sides with Transpose and `numpy.linalg.solve`

**Reference:** https://cs.stackexchange.com/questions/150877/finding-a-d-dimensional-hyperplane-containing-n-given-points.

**Limitation:**`np.linalg.solve` with "A.T @" does not work for overdetermined or underdetermined systems

In [3]:
def solve(lhs, rhs):
    try:
        result = np.linalg.solve(lhs, rhs)
        pretty_print_hyperplane(result)
    except:
        print("Solution cannot be found (overdetermined system)")

A_2x3 = np.array([
    [1, 2, 3],
    [4, 8, 12],
])
b_2x3 = np.ones(2)

A_3x3 = np.array([
    [1, 2, 3],
    [4, 5, 6],
    [7, 8, 9],
])
b_3x3 = np.ones(3)

A_4x3 = np.array([
    [1, 2, 3],
    [4, 5, 6],
    [7, 8, 9],
    [10, 11, 12]
])
b_4x3 = np.ones(4)

print('\033[4m' + '\033[1m' + "Without Multiplying by Transpose" + '\033[0m')
solve(A_2x3, b_2x3)
solve(A_3x3, b_3x3)
solve(A_4x3, b_4x3)
print()
print('\033[4m' + '\033[1m' + "Multiplying by Transpose:" + '\033[0m')
solve(A_2x3.T @ A_2x3, A_2x3.T @ b_2x3)
solve(A_3x3.T @ A_3x3, A_3x3.T @ b_3x3)
solve(A_4x3.T @ A_4x3, A_4x3.T @ b_4x3)

Without Multiplying by Transpose
Solution cannot be found (overdetermined system)
Solution cannot be found (overdetermined system)
Solution cannot be found (overdetermined system)

Multiplying by Transpose:
Solution cannot be found (overdetermined system)
-0.5625*a + 0.1250*b + 0.4375*c = 1.0
Solution cannot be found (overdetermined system)


##### Method 3: Solving Linear System using `numpy.linalg.lstsq`

**Limitation:** May be slower than `numpy.linalg.solve`

In [4]:
def solve(lhs, rhs):
    try:
        solution, residuals, rank, singular = np.linalg.lstsq(lhs, rhs)
        pretty_print_hyperplane(solution)
    except:
        print("Solution cannot be found (overdetermined system)")

A_2x3 = np.array([
    [1, 2, 3],
    [4, 8, 12],
])
b_2x3 = np.ones(2)

A_3x3 = np.array([
    [1, 2, 3],
    [4, 5, 6],
    [7, 8, 9],
])
b_3x3 = np.ones(3)

A_4x3 = np.array([
    [1, 2, 3],
    [4, 5, 6],
    [7, 8, 9],
    [10, 11, 12]
])
b_4x3 = np.ones(4)

solve(A_2x3, b_2x3)
solve(A_3x3, b_3x3)
solve(A_4x3, b_4x3)

0.0210*a + 0.0420*b + 0.0630*c = 1.0
-0.5000*a + 0.0000*b + 0.5000*c = 1.0
-0.5000*a + 0.0000*b + 0.5000*c = 1.0


##### Method 4: Solving Linear System using Pseudo-Inverse

**Limitation**: Less efficient than computing inverse

In [5]:
def solve(lhs, rhs):
    try:
        result = np.linalg.pinv(lhs) @ rhs
        pretty_print_hyperplane(result)
    except:
        print("Solution cannot be found by function")

A_2x3 = np.array([
    [1, 2, 3],
    [4, 8, 12],
])
b_2x3 = np.ones(2)

A_3x3 = np.array([
    [1, 2, 3],
    [4, 5, 6],
    [7, 8, 9],
])
b_3x3 = np.ones(3)

A_4x3 = np.array([
    [1, 2, 3],
    [4, 5, 6],
    [7, 8, 9],
    [10, 11, 12]
])
b_4x3 = np.ones(4)

solve(A_2x3, b_2x3)
solve(A_3x3, b_3x3)
solve(A_4x3, b_4x3)

0.0210*a + 0.0420*b + 0.0630*c = 1.0
-0.5000*a + 0.0000*b + 0.5000*c = 1.0
-0.5000*a + 0.0000*b + 0.5000*c = 1.0
